In [ ]:
# Libraries
import sys
sys.path.append('C:\Proyectos\Loteria\DataBase\Scrapping')
from scrapping import get_data
import pandas as pd
import numpy as np
from collections import Counter
from decimal import Decimal, getcontext

In [ ]:
# Functions
def count_skips(df, list_numbers):
    counts = {key: 0 for key in list_numbers}
    
    for columns in df:
        counter = 0
        for i in reversed(df[columns]):
            if not i:
                counter += 1
            else:
                counts[columns] = counter
                break
                
    return counts

def max_output(df):
    df = df.T
    max_numbers = df.sort_values(by=df.columns[0], ascending=False)
    return max_numbers

def year_hits(database, df_with_numbers, numbers_quantity):
    db_year = database['Dates'].dt.year
    year_history = pd.DataFrame(columns = ['Year', 'Number', 'Count'])
    
    for year in range(db_year.min(), db_year.max()+1):
        for number in numbers_quantity:
            filtered = df_with_numbers.loc[(db_year == year) & (df_with_numbers == number).any(axis=1), :]
            count = filtered.eq(number).sum().sum()
            year_history = pd.concat([year_history, pd.DataFrame({'Year': year, 'Number': number, 'Count': count}, index=[0])], ignore_index=True)
            
    return year_history

def get_hits(database, df_with_numbers, numbers_quantity):
    hits = year_hits(database, df_with_numbers, numbers_quantity).pivot_table(index='Year', columns='Number', values='Count', fill_value=0).sum().reset_index()
    hits = hits.rename(columns = {'Number': 'Numbers', 'Count': 'Hits'}).set_index('Numbers').T
    hits = hits.iloc[0].rename('Hits').to_frame()
    hits.index.name = None
    return hits.T

def total_average_hits(database, hits, numbers, is_star=False, aprox=False):
    divide = 2 if is_star else 5
    average_hits = hits.apply(lambda hits: hits / len(database) / divide)
    average_hits = average_hits.iloc[0].rename('Average').to_frame()
    average_hits.index.name = None
    getcontext().prec = 28
    
    if aprox:
        return Decimal(average_hits['Average'].sum() / numbers.max() + 0.001).quantize(Decimal('0.000001'))
    else:
        return Decimal(average_hits['Average'].sum() / numbers.max()).quantize(Decimal('0.000001'))

def minimal_hits(database, hits, numbers, average, is_star=False, aprox=False):
    min_hits = total_average_hits(database, hits, numbers, is_star, aprox)
    return float(min_hits) * hits.iloc[0, 0] / average.iloc[0, 0]

def average_hits(database, hits_data, numbers, is_star = False):
    divide = 2 if is_star else 5
    hits_by_num = hits_data.apply(lambda hits: hits / len(database) / divide)
    hits_by_num = hits_by_num.iloc[0].rename('Average').to_frame()
    hits_by_num.index.name = None
    averages = []
    
    for num in numbers:
        avg = hits_by_num.loc[num]['Average'].sum()
        averages.append(round(avg, 8))
        
    result = pd.DataFrame({'Numbers/Stars': numbers, 'Average_Hits': averages})
    result = result.T
    result.columns = result.iloc[0].astype(int)
    result = result[1:]
    return result

def natural_rotation(database, hits, numbers, data_average, index_start, index_end, is_star=False, aprox=False):
    average = total_average_hits(database, hits, numbers, is_star, aprox)
    m_hits = minimal_hits(database, hits, numbers, data_average, is_star, aprox)
    rotation = pd.DataFrame({'Hits': hits.iloc[0], 'Average_Numbers': data_average.iloc[0], 'Average': average, 'Hits_Needed': m_hits}, index = range(index_start, index_end + 1))
    rotation['Difference'] = rotation['Hits'] - rotation['Hits_Needed']
    return rotation

def get_rotations(database, hits, numbers, data_average, is_star=False):
    approx_rotations = []
    exact_rotations = []
    index_ranges = [(1, 25, '_low'), (26, 50, '_high')]
    
    for start, end, suffix in index_ranges:
        # exact rotations
        exact_rotation = natural_rotation(database, hits, numbers, data_average, start, end, is_star=is_star, aprox=False)
        exact_rotations.append(exact_rotation)
        # approx rotations
        approx_rotation = natural_rotation(database, hits, numbers, data_average, start, end, is_star=is_star, aprox=True)
        approx_rotations.append(approx_rotation)
        
    return tuple(approx_rotations + exact_rotations)

In [ ]:
# def year_hits(database, df_with_numbers, numbers_quantity):
#     db_year = database['Dates'].dt.year
    
#     def count_hits(year_number):
#         year, number = year_number
#         filtered = df_with_numbers.loc[(db_year == year) & (df_with_numbers == number).any(axis=1), :]
#         count = filtered.eq(number).sum().sum()
#         return pd.Series({'Year': year, 'Number': number, 'Count': count})
    
#     year_numbers = [(y, n) for y in range(db_year.min(), db_year.max()+1) for n in numbers_quantity]
#     year_history = pd.DataFrame(year_numbers, columns=['Year', 'Number'])
#     year_history = year_history.apply(count_hits, axis=1)
#     year_history = year_history.pivot_table(index='Year', columns='Number', values='Count', fill_value=0)
#     return year_history
# por probar

In [ ]:
# Load the data base and obtain the first DataFrame
db = pd.read_parquet('C:\Proyectos\Loteria\DataBase\db.parquet')
winning_numbers = db.iloc[:, 2:7]
winning_stars = db.iloc[:, 7:9]
total_numbers = np.arange(1, 51)
total_stars = np.arange(1, 13)

# Create a template DataFrame with all values set to False
skip_winners_bool = pd.DataFrame(False, columns = [str(i) for i in range(1, 51)], index = range(len(winning_numbers)))

# Fill in the True values
for e in range(1, 6):
    for i in range(1, 51):
        skip_winners_bool[f"{i}"] |= (winning_numbers[f"Nro{e}"] == i)

# Add an extra row, in order to compare the number that did not appear for the first time in the game history
d_0 = pd.DataFrame(columns = [str(i) for i in range(1, 51)], index = [0]).fillna(True)

# Create the final DataFrame
skip_winners = pd.concat([d_0, skip_winners_bool]).reset_index(drop = True)
del d_0
del skip_winners_bool

# Hits of numbers
numbers_hits = get_hits(db, winning_numbers, total_numbers)

# Hits of stars since draw 940, this is because of the change of rules. The star 12 was added in September 24th 2016
stars_filtered = db[(db['Sorteos'] > 940)].iloc[:, [0, 7, 8]]
stars_filtered_hits = get_hits(stars_filtered, stars_filtered, total_stars)

# Year History for numbers and stars
numbers_year_history = year_hits(db, winning_numbers, total_numbers)
stars_year_history = year_hits(db, stars_filtered, total_stars)
numbers_year_history = numbers_year_history.pivot_table(index = 'Year', columns = 'Number', values = 'Count', fill_value = 0)
stars_year_history = stars_year_history.pivot_table(index = 'Year', columns = 'Number', values = 'Count', fill_value = 0)
ny_mean = pd.DataFrame(numbers_year_history.mean(), columns=['Average']).T.rename(index={'0': 'Average'})
ny_median = pd.DataFrame(numbers_year_history.median(), columns=['Median']).T.rename(index={'0': 'Median'})
numbers_year_history = pd.concat([ny_median, ny_mean, numbers_year_history])

# Average of hits per numbers
numbers_average = average_hits(db, numbers_hits, total_numbers)

# Average of hits per star
stars_average = average_hits(stars_filtered, stars_filtered_hits, total_stars, is_star = True)

# Natural rotation of the numbers, using exact and aproximation values
aprox_rotation_BN_low, aprox_rotation_BN_high, exact_rotation_BN_low, exact_rotation_BN_high = get_rotations(db, numbers_hits, total_numbers, numbers_average, is_star=False)

In [ ]:
# It creates the list of draws, numbers and the dictionary to obtain the amount of skips per number if wins and looses.
draws = list(np.arange(1, len(skip_winners)))
numbers = [str(i) for i in range(1, 51)]
dicts = {draw: {key:[] for key in numbers} for draw in draws}

for e in draws:
    df = skip_winners.loc[:e]
    counts = count_skips(df, numbers)
    dicts[e].update(counts)

skip_numbers = pd.DataFrame(dicts).T
del draws
del numbers

In [ ]:
# Order the last draw for skips:
last_draw = sorted(dicts[len(skip_numbers)].items(), key = lambda x: x[1])
last_draw = pd.DataFrame({'Numero': [x[0] for x in last_draw], 'Skips': [int(x[1]) for x in last_draw]})
del dicts

# Select the last 12 draws
last_12_draws = np.arange(len(skip_numbers) - 12, len(skip_numbers) + 1)
sk_12 = skip_numbers.loc[last_12_draws]

# This establish the skips of the last 12 draws
skips = np.arange(0, 19)
aus_12 = [sk_12.loc[i - 1, str(column)] for i in last_12_draws[1:13] for column in sk_12 if sk_12.loc[i, str(column)] == 0]
counter_7 = Counter(aus_12[25:60])
counter_12 = Counter(aus_12)
last_7 = [counter_7.get(i, 0) for i in skips]
last_12 = [counter_12.get(i, 0) for i in skips]
skips_7_12 = pd.DataFrame({'7': last_7, '12': last_12})
del skips
del aus_12

In [ ]:
groups = [list(range(i, i + 10)) for i in range(1, 51, 10)]
group_names = [tuple(range(i, i + 10)) for i in range(1, 51, 10)]
results = {i: {group_name: sum([1 for num in row if num in group]) for group_name, group in zip(group_names, groups)} for i, row in winning_numbers.iterrows()}
results_df = pd.DataFrame.from_dict(results, orient = 'index')
results_df.columns = ['{}_to_{}'.format(i, i + 9) for i in range(1, 51, 10)]
sg_10 = results_df.iloc[-10:]
sg_5 = results_df.iloc[-5:]
groups_df = pd.DataFrame({'10_games': (sg_10 > 0).sum(), '5_games': (sg_5 > 0).sum()}).T

In [ ]:
db

In [ ]:
last_draw

In [ ]:
skips_7_12

In [ ]:
results_df

In [ ]:
groups_df

In [ ]:
numbers_hits

In [ ]:
numbers_year_history

In [ ]:
aprox_rotation_BN_low

In [ ]:
aprox_rotation_BN_high

In [ ]:
exact_rotation_BN_low

In [ ]:
exact_rotation_BN_high

In [ ]:
stars_filtered_hits

In [ ]:
stars_year_history